In [22]:
#IMPORT ALL REQUIRED MODULES
import json
import glob
import numpy as np
import pandas as pd
import csv
import string
import re
import math
import sklearn as sk
import nltk
import heapq
import matplotlib.pyplot as plt

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import seaborn as sns

In [24]:
#LOCATE EXACT LOCATION OF ALL JSON FILES IN KAGGLE LOCAL SETUP
train_files = glob.glob("../input/part-2/SmartLawDataset-main/json/*.json")

In [25]:
#EXTRACT INDIVIDUAL FILES AND GET THEM INTO SINGLE DATAFRAME
df_final=pd.DataFrame()
for file in train_files:
    with open(file) as f1: 
        data=json.load(f1) #ALL DATA FROM FILE LOADED INTO DATA
        df=pd.DataFrame([data]) #CREATE DATAFRAME FOR SINGLE DATA
        df_final=pd.concat([df_final,df]) #MERGE ALL INDIVIDUAL DATAFRAME INTO SINGLE DATAFRAME
#MAIN AIM IS TO GENERATE ML ALGO TO DETERMINE ARGUMENT TEXT AND ARGUMENT BY SO REMOVE REST
df_final=df_final.drop(['header','background','order','footer','annotationProcessingStage','annotationProcessingStageAnnotations','processedText'],axis=1)
#AXIS=1 REMOVE COLUMNS

In [26]:
#df_final1 is dataframe for argument text and argument by split into separate entities
header=['text','by']
filename = 'final_argument_by.csv'
data1=[]
with open(filename, 'w',encoding="UTF-8") as file:
    csvwriter = csv.writer(file) # 2. create a csvwriter object
    csvwriter.writerow(header) # 4. write the header
    for j in df_final['arguments']:
        for k in j:
            data=[]
            s=k['text'][3:]
            data.append(s.strip())
            data.append(k['argumentBy'])
            data1.append(data)
    csvwriter.writerows(data1) # 5. write the rest of the data
df_final1=pd.read_csv(filename)
#Use df_final1 dataframe only for further processes and not csv files to ensure encoding is in proper format

In [27]:
s =  set(string.punctuation)
for index, row in df_final1.iterrows():
    for x in row['text']:
        if x in s or re.search(r'-?\d+', x): 
            row['text']=row['text'].replace(x,"")
#Now we have proper dataset after removing unwanted punctuations etc

In [28]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df_final1['text'])
columns=count_vect.get_feature_names_out()
fin_x=pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names_out())

In [29]:
final_by=[]
for item in df_final1['by']:
    final_by.append(item) 
j=0
header=columns
filename = 'final_csv_1.csv'
data1=[]
with open(filename, 'w',encoding="UTF-8") as file:
    csvwriter = csv.writer(file) # 2. create a csvwriter object
    csvwriter.writerow(header) # 4. write the header
    for item in X_counts.toarray():
        data1.append(item)
    csvwriter.writerows(data1) # 5. write the rest of the data
final_bow=pd.read_csv(filename)
final_bow['By']=df_final1['by']
final_bow.head()
# final_bow is the bag of words dataframe along with argument by

In [30]:
final_bow = final_bow.dropna(axis=0)
# drop missing values if any

In [31]:
y=final_bow.By
features=header
x=final_bow[features]
# Setting up x and y coordinates
# x.head()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
# check the shape of X_train and X_test
X_train.shape, X_test.shape

In [33]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

In [34]:
y_pred = gnb.predict(X_test)
y_pred_train = gnb.predict(X_train)

In [35]:
print('Model accuracy score BoW-GNB: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [36]:
print(y_test.value_counts())
null_accuracy = (66/(36+66+20))
print('Null accuracy score: {0:0.4f}'. format(null_accuracy))

In [37]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

In [38]:
y_pred = svclassifier.predict(X_test)

In [39]:
print('Model accuracy score BoW-SVC: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [40]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(X_train, y_train)  

In [41]:
y_pred= classifier.predict(X_test) 

In [42]:
print('Model accuracy score BoW-KNN: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [43]:
def learn_curve(X,y,c):
    le = preprocessing.LabelEncoder() # Label encoding the target
    sc = preprocessing.StandardScaler() # Scaling the input features
    y = le.fit_transform(y)#Label Encoding the target
    log_reg = LogisticRegression(max_iter=200,random_state=11,C=c)
    lr = Pipeline(steps=(['scaler',sc],
                        ['classifier',log_reg]))
    
    
    cv = StratifiedKFold(n_splits=5,random_state=11,shuffle=True) # Creating a StratifiedKFold object with 5 folds
    cv_scores = cross_val_score(lr,X,y,scoring="accuracy",cv=cv) # Storing the CV scores (accuracy) of each fold
    lr.fit(X,y) # Fitting the model
    train_score = lr.score(X,y) # Scoring the model on train set
    #Building the learning curve
    train_size,train_scores,test_scores = learning_curve(estimator=lr,X=X,y=y,cv=cv,scoring="accuracy",random_state=11)
    train_scores = 1-np.mean(train_scores,axis=1)#converting the accuracy score to misclassification rate
    test_scores = 1-np.mean(test_scores,axis=1)#converting the accuracy score to misclassification rate
    lc = pd.DataFrame({"Training_size":train_size,"Training_loss":train_scores,"Validation_loss":test_scores}).melt(id_vars="Training_size")
    return {"cv_scores":cv_scores,
           "train_score":train_score,
           "learning_curve":lc}

In [44]:
lc = learn_curve(x,y,1)
print(f'Cross Validation Accuracies:\n{"-"*25}\n{list(lc["cv_scores"])}\n\n\
Mean Cross Validation Accuracy:\n{"-"*25}\n{np.mean(lc["cv_scores"])}\n\n\
Standard Deviation of Cross Validation Accuracy:\n{"-"*25}\n{np.std(lc["cv_scores"])}\n\n\
Training Accuracy:\n{"-"*15}\n{lc["train_score"]}\n\n')
sns.lineplot(data=lc["learning_curve"],x="Training_size",y="value",hue="variable")
plt.title("Learning Curve of Model")
plt.ylabel("Misclassification Rate/Loss");